In [1]:
import os
import subprocess
import cv2
import shutil

In [2]:
from __future__ import division
from PIL import Image

In [8]:
from contextlib import closing
import pymysql
from pymysql.cursors import DictCursor

In [9]:
from pydrive.auth import GoogleAuth

#gauth = GoogleAuth()
#gauth.LocalWebserverAuth() 

In [10]:
from pydrive.drive import GoogleDrive

drive = GoogleDrive(gauth)

In [11]:
def ListFolder(parent, folder):
    results = []
    file_list = []
    file_list = drive.ListFile({'q': "'%s' in parents and trashed=false" % parent}).GetList()
    for f in file_list:
        if f['mimeType']=='application/vnd.google-apps.folder': # if folder
            filelist.append({"id":f['id'],"title":f['title'],"list":ListFolder(f['id'])})
        elif f['mimeType']=='video/x-msvideo' and f['title'] not in  ["11480.avi", "11460.avi", "11539.avi", "11538.avi", "11537.avi", "11536.avi", "11535.avi", "11534.avi", "11533.avi", "11532.avi"]:
            file = drive.CreateFile({'id': f['id']})
            file.GetContentFile(folder + f['title'])
            results = add_video(f["title"], f["alternateLink"])
            for video in os.listdir(dir_main):
                if '.avi' in video:
                    os.makedirs("pict", exist_ok=True)
                    subprocess.call(['ffmpeg', '-i', video, '-vf', "select='isnan(prev_selected_t)+gte(t-prev_selected_t\,10)'", '-vsync', '0', '-an', '-frame_pts', '1', 
                             'pict/%d.jpg'])
                    #os.makedirs(dir_main + os.path.splitext(video)[0] + '/', exist_ok=True)
                    os.rename(dir_main + video, subdir + video)
                    os.rename(subdir, dir_main + os.path.splitext(video)[0] + '/')
                    for vid in os.listdir(os.path.splitext(video)[0]):
                        if '.avi' in vid:
                            for element in results:
                                    if vid == element["video_name"]:
                                        search_similar(vid, element["id"], dir_main + vid.split(".")[0] + '/', dir_main)       
                    delete_dir(video.split(".")[0])


In [12]:
def add_video(video_name, video_url):
    with closing(pymysql.connect(
        host='localhost',
        user='Dina',
        password='hsehse',
        db='miem',
        charset='utf8mb4',
        cursorclass=DictCursor
    )) as connection:
        with connection.cursor() as cursor:
            query = """
            INSERT INTO
                video_table (video_name, video_url, video_fps)
            VALUES
                ( %(video_name)s, %(video_url)s , %(video_fps)s)
            """
            data = [{
                'video_name': video_name,
                'video_url': video_url,
                'video_fps': cv2.VideoCapture(subdir + video_name).get(cv2.CAP_PROP_FPS)
            }]
            cursor.executemany(query, data)
            connection.commit()
            cursor.execute("SELECT * FROM video_table WHERE video_url=%s LIMIT 1", video_url)
            results = cursor.fetchall() 
            connection.commit() 
            return(results)

In [13]:
def search_hash(video_id, hash_curr, filename, tolerance):
    with closing(pymysql.connect(
        host='localhost',
        user='Dina',
        password='hsehse',
        db='miem',
        charset='utf8mb4',
        cursorclass=DictCursor
    )) as connection:
        with connection.cursor() as cursor:
            cursor.execute("SELECT * FROM hash_table WHERE video_id NOT LIKE %s", video_id)
            results = cursor.fetchall()
            connection.commit()
            for res in results:
                #print(res)
                if hashes_are_similar(res["hash"], hash_curr, tolerance):
                    write_logs(video_id, filename, res["video_id"], res["timecode"])

In [14]:
def write_logs(video1, timecode1, video2, timecode2):
    with closing(pymysql.connect(
        host='localhost',
        user='Dina',
        password='hsehse',
        db='miem',
        charset='utf8mb4',
        cursorclass=DictCursor
    )) as connection:
        with connection.cursor() as cursor:
            query = """
            INSERT INTO
                logs (video1_id, timecode_1, video2_id, timecode_2)
            VALUES
                ( %(name1)s, %(time1)s, %(name2)s, %(time2)s )
            """
            data = [{
                'name1': video1,
                'time1': timecode1,
                'name2': video2,
                'time2': timecode2
            }]
            cursor.executemany(query, data)
            connection.commit()

In [15]:
def add_hash(video_id, timecode, video_hash):
    with closing(pymysql.connect(
        host='localhost',
        user='Dina',
        password='hsehse',
        db='miem',
        charset='utf8mb4',
        cursorclass=DictCursor
    )) as connection:
        with connection.cursor() as cursor:
            query = """
            INSERT INTO
                hash_table (video_id, timecode, hash)
            VALUES
                ( %(video_id)s, %(timecode)s, %(hash)s )
            """
            data = [{
                'video_id': video_id,
                'timecode': timecode,
                'hash': video_hash
            }]
            cursor.executemany(query, data)
            connection.commit()

In [16]:
def hash_distance(left_hash, right_hash):
    """Compute the hamming distance between two hashes"""
    if len(left_hash) != len(right_hash):
        raise ValueError('Hamming distance requires two strings of equal length')

    return sum(map(lambda x: 0 if x[0] == x[1] else 1, zip(left_hash, right_hash)))


def hashes_are_similar(left_hash, right_hash, tolerance=0):
    """
    Return True if the hamming distance between
    the image hashes are less than the given tolerance.
    """
    return hash_distance(left_hash, right_hash) <= tolerance


def average_hash(image_path, hash_size=8):
    """ Compute the average hash of the given image. """
    with open(image_path, 'rb') as f:
        # Open the image, resize it and convert it to black & white.
        image = Image.open(f).resize((hash_size, hash_size), Image.ANTIALIAS).convert('L')
        pixels = list(image.getdata())

    avg = sum(pixels) / len(pixels)

    # Compute the hash based on each pixels value compared to the average.
    bits = "".join(map(lambda pixel: '1' if pixel > avg else '0', pixels))
    hashformat = "0{hashlength}x".format(hashlength=hash_size ** 2 // 4)
    return int(bits, 2).__format__(hashformat)


def distance(image_path, other_image_path):
    """ Compute the hamming distance between two images"""
    image_hash = average_hash(image_path)
    other_image_hash = average_hash(other_image_path)

    return hash_distance(image_hash, other_image_hash)


def is_look_alike(image_path, other_image_path, tolerance=0):
    image_hash = average_hash(image_path)
    other_image_hash = average_hash(other_image_path)

    return hashes_are_similar(image_hash, other_image_hash, tolerance)

In [17]:
def search_similar(video, video_id, subdir, dir_main):
    name = ''

    for filename in os.listdir(subdir):
        if '.jpg' in filename:
            file = filename.split(".")[0]
            hash_curr = average_hash(subdir + '/' + file + ".jpg")
            search_hash(video_id, hash_curr, file, tolerance = 0)
            add_hash(video_id, file, hash_curr)
                            
def delete_dir(folder):
    print(dir_main + folder)
    shutil.rmtree(dir_main + folder, ignore_errors=True)

In [18]:
def main():
    ListFolder('0B-FXLXAQElhrfjNiX0ZYMWFGWUlRenRBVFRlTldvbDFkdnVTWUVnOHJDZE1UbWpYM1JrZkU', dir_main)
    #print(database)

In [20]:
#print('Введите путь к файлам')
dir_main = 'C:/Users/dfkhasanova/Desktop/Учеба/NIR/' #input()
subdir = dir_main + 'pict/'
sub = subdir + '/%d.jpg'
main()

NameError: name 'drive' is not defined

In [115]:
os.rename(subdir, dir_main + os.path.splitext(video)[0] + '/')
for vid in os.listdir(os.path.splitext(video)[0]):
    if '.avi' in vid:
        search_similar(vid, "18", dir_main + vid.split(".")[0] + '/', dir_main)       
delete_dir(video.split(".")[0])

C:/Users/dfkhasanova/Desktop/Учеба/NIR/11534


In [112]:
video = "11534.avi"

In [6]:
import mysql.connector
from mysql.connector import Error

def connect():
    """ Connect to MySQL database """
    try:
        conn = mysql.connector.connect(host='localhost',
                                       database='python_mysql',
                                       user='root',
                                       password='secret')
        if conn.is_connected():
            print('Connected to MySQL database')

    except Error as e:
        print(e)

    finally:
        conn.close()

ModuleNotFoundError: No module named 'mysql'

In [7]:
import mysql.connector
from mysql.connector import Error
from mysql.connector import errorcode

try:
    connection = mysql.connector.connect(host='localhost',
                                         database='electronics',
                                         user='root',
                                         password='pynative@#29')
    mySql_insert_query = """INSERT INTO Laptop (Id, Name, Price, Purchase_date) 
                           VALUES 
                           (10, 'Lenovo ThinkPad P71', 6459, '2019-08-14') """

    cursor = connection.cursor()
    cursor.execute(mySql_insert_query)
    connection.commit()
    print(cursor.rowcount, "Record inserted successfully into Laptop table")
    cursor.close()

except mysql.connector.Error as error:
    print("Failed to insert record into Laptop table {}".format(error))

finally:
    if (connection.is_connected()):
        connection.close()
        print("MySQL connection is closed")

OperationalError: (2003, "Can't connect to MySQL server on '192.168.5.134' (timed out)")